In [ ]:
# =============================================================================
# Training and plotting of final ANN
# =============================================================================

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

# empty list to append metric values
mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# 3 months
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size = 0.15, shuffle=False)

# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor

# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40, 
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 15))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.3))
    for layer in range(n_hidden):
        model.add(Dense(units = n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.3))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

tscv = TimeSeriesSplit(n_splits = 6)

hist_list = pd.DataFrame()
count = 1
    
regressor = regressor_tunning()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    hist = regressor.fit(X_train_split, y_train_split, batch_size = 10, epochs = 100)
    hist_list = hist_list.append(hist.history, ignore_index = True)
    print(count)
    count = count + 1

a = []
b = []

for i in range(len(hist_list.mse)):
    a.append(np.mean(hist_list.mse[i]))
    b.append(np.mean(hist_list.mae[i]))

mse_cv.append(np.mean(a))
mae_cv.append(np.mean(b))

# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525

rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)

# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================

y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


# smal adjustment
y_test.replace(0, 0.0001,inplace = True)


# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)

rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# Metric evaluation on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)

rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)
# Save


results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})



C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Epoch 1/100
72/72 [==============================] - 0s 2ms/step - loss: 17060.1055 - mse: 17060.1055 - mae: 113.7836
Epoch 2/100
72/72 [==============================] - 0s 1ms/step - loss: 13981.5879 - mse: 13981.5879 - mae: 98.5571
Epoch 3/100
72/72 [==============================] - 0s 942us/step - loss: 6235.8999 - mse: 6235.8999 - mae: 47.2687
Epoch 4/100
72/72 [==============================] - 0s 1ms/step - loss: 6032.9775 - mse: 6032.9775 - mae: 43.7270
Epoch 5/100
72/72 [==============================] - 0s 974us/step - loss: 6120.3101 - mse: 6120.3101 - mae: 44.2040
Epoch 6/100
72/72 [==============================] - 0s 1ms/step - loss: 5592.5923 - mse: 5592.5923 - mae: 41.2733
Epoch 7/100
72/72 [==============================] - 0s 1ms/step - loss: 6199.6396 - mse: 6199.6401 - mae: 43.9298
Epoch 8/100
72/72 [==============================] - 0s 1ms/step - loss: 5934.0181 - mse: 5934.0181 - mae: 43.6502
Epoch 9/100
72/72 [==============================] - 0s 1ms/step - loss

72/72 [==============================] - 0s 1ms/step - loss: 4989.6489 - mse: 4989.6489 - mae: 35.9956
Epoch 72/100
72/72 [==============================] - 0s 1ms/step - loss: 5105.1748 - mse: 5105.1748 - mae: 36.1319
Epoch 73/100
72/72 [==============================] - 0s 1ms/step - loss: 4973.3823 - mse: 4973.3823 - mae: 34.1595
Epoch 74/100
72/72 [==============================] - 0s 1ms/step - loss: 5069.6411 - mse: 5069.6411 - mae: 35.4200
Epoch 75/100
72/72 [==============================] - 0s 1ms/step - loss: 5159.9736 - mse: 5159.9736 - mae: 36.1684
Epoch 76/100
72/72 [==============================] - 0s 1ms/step - loss: 4921.0146 - mse: 4921.0146 - mae: 34.6088
Epoch 77/100
72/72 [==============================] - 0s 1ms/step - loss: 4922.0522 - mse: 4922.0527 - mae: 35.1574
Epoch 78/100
72/72 [==============================] - 0s 2ms/step - loss: 4790.2354 - mse: 4790.2354 - mae: 34.6886
Epoch 79/100
72/72 [==============================] - 0s 1ms/step - loss: 4946.2222 -

143/143 [==============================] - 0s 2ms/step - loss: 3469.9297 - mse: 3469.9297 - mae: 33.6522
Epoch 41/100
143/143 [==============================] - 0s 2ms/step - loss: 3516.5161 - mse: 3516.5161 - mae: 33.3097
Epoch 42/100
143/143 [==============================] - 0s 2ms/step - loss: 3488.9541 - mse: 3488.9541 - mae: 34.0080
Epoch 43/100
143/143 [==============================] - 0s 1ms/step - loss: 3546.0378 - mse: 3546.0378 - mae: 33.8979
Epoch 44/100
143/143 [==============================] - 0s 1ms/step - loss: 3469.5737 - mse: 3469.5737 - mae: 33.8177
Epoch 45/100
143/143 [==============================] - 0s 1ms/step - loss: 3606.6162 - mse: 3606.6162 - mae: 33.5928
Epoch 46/100
143/143 [==============================] - 0s 1ms/step - loss: 3534.4785 - mse: 3534.4785 - mae: 33.7181
Epoch 47/100
143/143 [==============================] - 0s 1ms/step - loss: 3581.5635 - mse: 3581.5635 - mae: 33.8101
Epoch 48/100
143/143 [==============================] - 0s 1ms/step -

214/214 [==============================] - 0s 2ms/step - loss: 2753.0112 - mse: 2753.0112 - mae: 31.1078
Epoch 10/100
214/214 [==============================] - 0s 2ms/step - loss: 2913.9653 - mse: 2913.9653 - mae: 31.2655
Epoch 11/100
214/214 [==============================] - 0s 2ms/step - loss: 2838.4399 - mse: 2838.4399 - mae: 31.2799
Epoch 12/100
214/214 [==============================] - 0s 2ms/step - loss: 2790.1333 - mse: 2790.1333 - mae: 30.6106
Epoch 13/100
214/214 [==============================] - 0s 1ms/step - loss: 2824.7852 - mse: 2824.7852 - mae: 30.8591
Epoch 14/100
214/214 [==============================] - 0s 1ms/step - loss: 2739.9238 - mse: 2739.9238 - mae: 30.4355
Epoch 15/100
214/214 [==============================] - 0s 1ms/step - loss: 2877.6843 - mse: 2877.6843 - mae: 31.3201
Epoch 16/100
214/214 [==============================] - 0s 2ms/step - loss: 2817.3479 - mse: 2817.3479 - mae: 30.0639
Epoch 17/100
214/214 [==============================] - 0s 2ms/step -

214/214 [==============================] - 0s 2ms/step - loss: 2701.9980 - mse: 2701.9980 - mae: 29.5268
Epoch 79/100
214/214 [==============================] - ETA: 0s - loss: 2799.8098 - mse: 2799.8098 - mae: 29.44 - ETA: 0s - loss: 2701.5261 - mse: 2701.5261 - mae: 29.59 - 0s 2ms/step - loss: 2622.5530 - mse: 2622.5530 - mae: 29.6459
Epoch 80/100
214/214 [==============================] - 0s 2ms/step - loss: 2719.0120 - mse: 2719.0120 - mae: 29.9963
Epoch 81/100
214/214 [==============================] - 0s 2ms/step - loss: 2745.2139 - mse: 2745.2139 - mae: 30.1645
Epoch 82/100
214/214 [==============================] - 0s 2ms/step - loss: 2629.1155 - mse: 2629.1155 - mae: 29.5980
Epoch 83/100
214/214 [==============================] - 0s 1ms/step - loss: 2681.2598 - mse: 2681.2598 - mae: 29.6297
Epoch 84/100
214/214 [==============================] - 0s 1ms/step - loss: 2581.2173 - mse: 2581.2168 - mae: 29.2377
Epoch 85/100
214/214 [==============================] - 0s 1ms/step - l

285/285 [==============================] - 0s 1ms/step - loss: 2194.8596 - mse: 2194.8596 - mae: 28.0960
Epoch 46/100
285/285 [==============================] - 0s 1ms/step - loss: 2331.4785 - mse: 2331.4785 - mae: 28.4789
Epoch 47/100
285/285 [==============================] - 0s 1ms/step - loss: 2327.6880 - mse: 2327.6882 - mae: 28.6604
Epoch 48/100
285/285 [==============================] - 0s 1ms/step - loss: 2293.0647 - mse: 2293.0647 - mae: 28.7744
Epoch 49/100
285/285 [==============================] - 0s 1ms/step - loss: 2298.5645 - mse: 2298.5645 - mae: 28.2181
Epoch 50/100
285/285 [==============================] - 0s 1ms/step - loss: 2316.3743 - mse: 2316.3743 - mae: 28.5095
Epoch 51/100
285/285 [==============================] - 0s 1ms/step - loss: 2280.7861 - mse: 2280.7861 - mae: 28.3629
Epoch 52/100
285/285 [==============================] - 0s 1ms/step - loss: 2286.2493 - mse: 2286.2493 - mae: 28.1522
Epoch 53/100
285/285 [==============================] - 0s 1ms/step -

356/356 [==============================] - 1s 2ms/step - loss: 2632.6611 - mse: 2632.6611 - mae: 28.0302
Epoch 15/100
356/356 [==============================] - 0s 1ms/step - loss: 2621.5266 - mse: 2621.5266 - mae: 28.0877
Epoch 16/100
356/356 [==============================] - 0s 1ms/step - loss: 2631.5820 - mse: 2631.5820 - mae: 28.0399
Epoch 17/100
356/356 [==============================] - 0s 1ms/step - loss: 2634.5439 - mse: 2634.5439 - mae: 27.5645
Epoch 18/100
356/356 [==============================] - 0s 1ms/step - loss: 2632.3262 - mse: 2632.3262 - mae: 27.9378
Epoch 19/100
356/356 [==============================] - 0s 1ms/step - loss: 2638.9734 - mse: 2638.9734 - mae: 27.8925
Epoch 20/100
356/356 [==============================] - 0s 1ms/step - loss: 2620.4590 - mse: 2620.4590 - mae: 28.0072
Epoch 21/100
356/356 [==============================] - 0s 1ms/step - loss: 2605.9895 - mse: 2605.9895 - mae: 27.8797
Epoch 22/100
356/356 [==============================] - 0s 1ms/step -

In [6]:
results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,30.939864,21.999466,66.545431,55.02067,20.851979,17.028997


In [7]:
results.to_csv('Results_100epochs_10batch_6splits_0.3_dropout')